In [124]:
# Download libraries for processing XML and HTML with Python

!pip install lxml

import lxml.html as lh

import requests

import pandas as pd

In [125]:
# Retrieve/download the file from the web-site

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Check the document on the web-site.
# All the table data ( headers, postal codes) are stored between the HTML tags

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of each row, it must have 3 columns
[len(T) for T in tr_elements]

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 12,
 18]

In [126]:
print('\n')
print('# Rows loaded into a list after download :',len(tr_elements))
print('\n')



# Rows loaded into a list after download : 185




In [127]:
#Create empty list  to simply generate the header information
# tr_elements[0]  contains the column headers
# List 'col' will simply be a list of 3 column headers, and an empty list for each header.

col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
print('\n')
print('Header of list: ')
print(col)
print('\n')



Header of list: 
[('Postal Code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]




In [128]:
#Since the first row is the header, data is stored on the second row onwards
# List 'col' is now appended with values for each of the 3 columns

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
     
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
                
        #If row is not of size 3, the //tr data is not from our table of postal codes
        # This step will filter out any unwanted data such as another table on the web page
    
        if len(T)!=3:
           break
             
        if i==0:
            # If Postal Code is empty, then data is not from the table of postal codes
            if len(data.strip())==0:
                break
        
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        
        #Increment i for the next column
        i+=1

In [129]:
# Checking the final list and verifying rows in each column
print('\n')
print('Checking the final list and verifying rows in each column :')
[len(C) for (title,C) in col]



Checking the final list and verifying rows in each column :


[180, 180, 180]

In [130]:
# Converting the list into a data frame

Dict={title:column for (title,column) in col}

df=pd.DataFrame(Dict)

In [131]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [132]:
df.shape

(180, 3)

In [133]:
# Check the tail of the data frame
df.tail()

,Postal Code\n,Borough\n,Neighborhood\n
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z\n,Not assigned\n,\n


In [134]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

# Count of rows with 'Borough' that is 'Not assigned'

print('\n')
print('Count of rows for each value of Boroughs  :\n', df['Borough\n'].value_counts())
print('\n')
df_Borough_To_drop = df.loc[df['Borough\n'] == 'Not assigned\n']
print('\n')
print('Boroughs to drop due to --Not Assigned-- values :')
print('\n')
print(df_Borough_To_drop)

# Drop rows with ''Borough'' that is ''Not assigned'', and then reset the index

df.drop(df.loc[df['Borough\n'] == 'Not assigned\n'].index, inplace=True)

df = df.reset_index(drop=True)
print('\n')
print('Size of dataframe after removing  --Borough-- that is --Not assigned--')
print('# Rows, Columns',df.shape)
print('\n')



Count of rows for each value of Boroughs  :
 Not assigned\n        77
North York\n          24
Downtown Toronto\n    19
Scarborough\n         17
Etobicoke\n           12
Central Toronto\n      9
West Toronto\n         6
East York\n            5
East Toronto\n         5
York\n                 5
Mississauga\n          1
Name: Borough\n, dtype: int64




Boroughs to drop due to --Not Assigned-- values :


    Postal Code\n       Borough\n Neighborhood\n
0           M1A\n  Not assigned\n             \n
1           M2A\n  Not assigned\n             \n
7           M8A\n  Not assigned\n             \n
10          M2B\n  Not assigned\n             \n
15          M7B\n  Not assigned\n             \n
..            ...             ...            ...
174         M4Z\n  Not assigned\n             \n
175         M5Z\n  Not assigned\n             \n
176         M6Z\n  Not assigned\n             \n
177         M7Z\n  Not assigned\n             \n
179         M9Z\n  Not assigned\n             \n

[77

In [135]:
# Remove the extraneous characters'\n' in every row, column 

df['Postal Code'] = df['Postal Code\n'].str.rstrip('\n')
df['Borough'] = df['Borough\n'].str.rstrip('\n')
df['Neighborhood'] = df['Neighborhood\n'].str.rstrip('\n')

df_new = df[['Postal Code','Borough','Neighborhood']]
df_new.head()
print('\n')
print ('Number of rows columns in the revised data frame', df_new.shape)
print('\n')



Number of rows columns in the revised data frame (103, 3)




In [136]:
# Check for rows where column Neighborhood has value has 'Not assigned'

df_Neighbor_NA = df_new.loc[df['Neighborhood'] == 'Not assigned']

#  Display the data frame that shows list of rows where 'Neighborhood' equals 'Not assigned'
print('\n')
print('Do any rows exist with no values for column Neighborhood ? ',df_Neighbor_NA)
print('\n')



Do any rows exist with no values for column Neighborhood ?  Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []




In [137]:
# Final data frame size after scrapping data

print('\n')
print('Rows and Columns of the final data frame : ', df_new.shape)
print('\n')



Rows and Columns of the final data frame :  (103, 3)




In [ ]:
# Check the data to ensure the right records are displayed

In [138]:
df_new.iloc[0:9]

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"


In [123]:
df_new.iloc[100:105]

,Postal Code,Borough,Neighborhood
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [ ]:
# Importing Longitude and Latitude for each postal codes

In [139]:
import pandas as pd
import numpy as np

In [140]:
filename = 'http://cocl.us/Geospatial_data'

In [141]:
# Read the contents of the file into a data frame, and assign column headers in the data fram

headers = ['Postal Code','Latitude','Longitude']

df_pcode_long_lat = pd.read_csv(filename, names=headers)

# Display the first 5 rows of the data frame

df_pcode_long_lat.head(5)

,Postal Code,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [145]:
# Check the columns of 2 dataframes to merge

# Dataframe with Postal code, Borough, Neighborr
print(df_new.columns)
print('\n')
print('Rows/Columns of PCode/Bor/Neigh :', df_new.shape)
print('\n')
# Dataframe with Postal code, Longitude, Latitude
print(df_pcode_long_lat.columns)
print('\n')
print('Rows/Columns of PCode/Lat/Lng :', df_pcode_long_lat.shape)

Index(['Postal Code', 'Borough', 'Neighborhood'], dtype='object')


Rows/Columns of PCode/Bor/Neigh : (103, 3)


Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


Rows/Columns of PCode/Lat/Lng : (104, 3)


In [149]:
df_pcode_long_lat.tail()

,Postal Code,Latitude,Longitude
99,M9N,43.706876,-79.5181884
100,M9P,43.696319,-79.5322424
101,M9R,43.6889054,-79.5547244
102,M9V,43.7394164,-79.5884369
103,M9W,43.7067483,-79.5940544


In [147]:
#Merge the 2 dataframes

merged_PCode = pd.merge(left=df_new, right=df_pcode_long_lat, left_on='Postal Code', right_on='Postal Code')


# What's the size of the output data?
merged_PCode.shape


(103, 5)

In [148]:
# Display data frame with Postal Code, Borough, Neighborhood, Latitude, Longitude
merged_PCode

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.4647633
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6536536,-79.5069436
99,M4Y,Downtown Toronto,Church and Wellesley,43.6658599,-79.3831599
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6627439,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6362579,-79.4985091
